In [27]:
from alpaca_trade_api.rest import REST, TimeFrame
import time

# ตั้งค่า API Key และ Secret สำหรับ Paper Account
API_KEY = "PK6JC4NIPDDSHGSSEMUX"  # ใส่ API Key ของคุณ
SECRET_KEY = "YwbBplZK6GLawJuoFm0g4MJbh0rY3JlTnx8kpzQa"  # ใส่ Secret Key ของคุณ
BASE_URL = "https://paper-api.alpaca.markets" # URL สำหรับ Paper Account

# สร้างการเชื่อมต่อกับ Alpaca
api = REST(API_KEY, SECRET_KEY, BASE_URL)

# ตรวจสอบสถานะบัญชี
account = api.get_account()
print(f"Account status: {account.status}")  # แสดงสถานะบัญชี


Account status: ACTIVE


In [2]:
# ฟังก์ชันสำหรับซื้อหุ้น
def buy_stock(symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='buy',
            type='market',
            time_in_force='gtc'  # Good Till Cancelled
        )
        print(f"Order placed to buy {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error buying stock: {e}")

# ฟังก์ชันสำหรับขายหุ้น
def sell_stock(symbol, qty):
    try:
        api.submit_order(
            symbol=symbol,
            qty=qty,
            side='sell',
            type='market',
            time_in_force='gtc'
        )
        print(f"Order placed to sell {qty} shares of {symbol}")
    except Exception as e:
        print(f"Error selling stock: {e}")


In [24]:
def trade_strategy(symbol, buy_price, sell_price):
    while True:
        # ดึงข้อมูลราคาหุ้นล่าสุด
        barset = api.get_bars(symbol, TimeFrame.Minute, limit=1)
        price=barset[0].c
        if barset:
            for bar in barset:
                print(f"Time: {bar.t}, Open: {bar.o}, High: {bar.h}, Low: {bar.l}, Close: {bar.c}, Volume: {bar.v}")

        # กลยุทธ์การซื้อขาย
        if price <= buy_price:
            print(f"Buying {symbol} at {price}")
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='buy',
                type='market',
                time_in_force='gtc'
            )
        elif price >= sell_price:
            print(f"Selling {symbol} at {price}")
            api.submit_order(
                symbol=symbol,
                qty=1,
                side='sell',
                type='market',
                time_in_force='gtc'
            )
        time.sleep(1) 


In [31]:
# เริ่มต้นกลยุทธ์สำหรับหุ้น AAPL
trade_strategy(symbol="AAPL", buy_price=247.75, sell_price=250)


Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706
Time: 2024-12-19 04:00:00-05:00, Open: 247.89, High: 247.89, Low: 247.75, Close: 247.79, Volume: 2706


KeyboardInterrupt: 

In [15]:
symbol="AAPL"
barset = api.get_bars(symbol, TimeFrame.Minute, limit=1)
print(f"Raw data for {symbol}: {barset}")


Raw data for AAPL: []


In [30]:
import asyncio
from alpaca_trade_api.stream import Stream
import nest_asyncio
nest_asyncio.apply()



base_url = "https://paper-api.alpaca.markets"
data_url = "wss://stream.data.alpaca.markets/v2/sip"  # ใช้ SIP สำหรับข้อมูลเรียลไทม์

symbol = "AAPL"  # หุ้นที่ต้องการติดตาม

async def quote_handler(data):
    """แสดงข้อมูลราคาล่าสุด"""
    print(f"Time: {data.timestamp}, Symbol: {data.symbol}, Bid: {data.bid_price}, Ask: {data.ask_price}")

async def main():
    stream = Stream(api_key, api_secret, base_url=data_url, raw_data=True)
    stream.subscribe_quotes(quote_handler, symbol)  # สมัครรับข้อมูลเรียลไทม์
    await stream.run()

# ตรวจสอบว่ามี event loop อยู่แล้วหรือไม่
if __name__ == "__main__":
    try:
        loop = asyncio.get_running_loop()
    except RuntimeError:  # ไม่มี event loop ที่กำลังทำงาน
        loop = asyncio.new_event_loop()
        asyncio.set_event_loop(loop)

    # รันฟังก์ชัน main
    loop.run_until_complete(main())


error during websocket communication: auth failed
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 251, in _run_forever
    await self._start_ws()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 105, in _start_ws
    await self._auth()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 99, in _auth
    raise ValueError(msg[0].get('msg', 'auth failed'))
ValueError: auth failed
error during websocket communication: auth failed
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 251, in _run_forever
    await self._start_ws()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 105, in _start_ws
    await self._auth()
  File "/opt/anaconda3/lib/python3.11/site-packages/alpaca_trade_api/stream.py", line 99, in _auth
    raise ValueError(msg[0].get

keyboard interrupt, bye


/opt/anaconda3/lib/python3.11/ast.py:50: RuntimeWarning: coroutine 'main' was never awaited
  return compile(source, filename, mode, flags,


TypeError: object NoneType can't be used in 'await' expression

In [21]:
import yfinance as yf

# กำหนดสัญลักษณ์หุ้น (ตัวอย่าง: AAPL - Apple)
symbol = "NVDA"

# ดึงข้อมูลย้อนหลัง 10 วัน
data = yf.download(symbol, period="11d", interval="1d")

# แสดงผลข้อมูล
print(data)



[*********************100%***********************]  1 of 1 completed

Price            Close        High         Low        Open     Volume
Ticker            NVDA        NVDA        NVDA        NVDA       NVDA
Date                                                                 
2025-01-16  133.570007  138.750000  133.490005  138.639999  209235600
2025-01-17  137.710007  138.500000  135.460007  136.690002  201188800
2025-01-21  140.830002  141.830002  137.089996  139.160004  197749000
2025-01-22  147.070007  147.789993  143.669998  144.660004  237651400
2025-01-23  147.220001  147.229996  143.720001  145.050003  155915500
2025-01-24  142.619995  148.970001  141.880005  148.369995  234657600
2025-01-27  118.419998  128.399994  116.699997  124.800003  818830900
2025-01-28  128.990005  129.000000  116.250000  121.809998  579666400
2025-01-29  123.699997  126.889999  120.050003  126.500000  467120600
2025-01-30  124.650002  125.000000  118.099998  123.099998  392925500
2025-01-31  120.070000  127.849998  119.190002  123.779999  388161100


In [17]:
import pandas as pd
pd.DataFrame(data)

Price,Close,High,Low,Open,Volume
Ticker,NVDA,NVDA,NVDA,NVDA,NVDA
Date,,,,,
2025-01-17,137.710007,138.500000,135.460007,136.690002,201188800
2025-01-21,140.830002,141.830002,137.089996,139.160004,197749000
2025-01-22,147.070007,147.789993,143.669998,144.660004,237651400
2025-01-23,147.220001,147.229996,143.720001,145.050003,155915500
2025-01-24,142.619995,148.970001,141.880005,148.369995,234657600
2025-01-27,118.419998,128.399994,116.699997,124.800003,818830900
2025-01-28,128.990005,129.000000,116.250000,121.809998,579666400
2025-01-29,123.699997,126.889999,120.050003,126.500000,467120600


In [22]:
data["Close"]["NVDA"]

Date
2025-01-16    133.570007
2025-01-17    137.710007
2025-01-21    140.830002
2025-01-22    147.070007
2025-01-23    147.220001
2025-01-24    142.619995
2025-01-27    118.419998
2025-01-28    128.990005
2025-01-29    123.699997
2025-01-30    124.650002
2025-01-31    120.070000
Name: NVDA, dtype: float64

In [15]:
import numpy as np
np.array(data["Close"]["NVDA"])

array([137.71000671, 140.83000183, 147.07000732, 147.22000122,
       142.61999512, 118.41999817, 128.99000549, 123.69999695,
       124.65000153, 120.06999969])

In [18]:
k=pd.DataFrame()

In [19]:
k["Close"]=np.array(data["Close"]["NVDA"])

In [20]:
k

,Close
0,137.710007
1,140.830002
2,147.070007
3,147.220001
4,142.619995
5,118.419998
6,128.990005
7,123.699997
8,124.650002
9,120.070000


In [23]:
import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

st.title("NVDA Stock Price Prediction")

import yfinance as yf

# กำหนดสัญลักษณ์หุ้น (ตัวอย่าง: AAPL - Apple)
symbol = "NVDA"

# ดึงข้อมูลย้อนหลัง 10 วัน
data = yf.download(symbol, period="11d", interval="1d")

df=pd.DataFrame()
df["Open"]=np.array(data["Open"]["NVDA"])
df["High"]=np.array(data["High"]["NVDA"])
df["Low"]=np.array(data["Low"]["NVDA"])
df["Close"]=np.array(data["Close"]["NVDA"])
df["Volume"]=np.array(data["Volume"]["NVDA"])

df["diff_n_o"]=[None]+[np.log(df["Open"][i+1])-np.log(df["Open"][i]) for i in (range(len(df)-1))]
df["diff_n_h"]=[None]+[np.log(df["High"][i+1])-np.log(df["High"][i]) for i in (range(len(df)-1))]
df["diff_n_c"]=[None]+[np.log(df["Close"][i+1])-np.log(df["Close"][i]) for i in (range(len(df)-1))]
df["diff_n_v"]=[None]+[np.log(df["Volume"][i+1])-np.log(df["Volume"][i]) for i in (range(len(df)-1))]
df["diff_n_l"]=[None]+[np.log(df["Low"][i+1])-np.log(df["Low"][i]) for i in (range(len(df)-1))]
df["diff_n_CO"]=[None]+[np.log(df["Close"][i+1])-np.log(df["Open"][i+1]) for i in (range(len(df)-1))]
data2=df.dropna()
Xl1=data2[["diff_n_o","diff_n_l","diff_n_h","diff_n_c","diff_n_v","diff_n_CO"]]

2025-02-01 14:58:19.350 
  command:

    streamlit run /opt/anaconda3/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
[*********************100%***********************]  1 of 1 completed


In [24]:
from keras.saving import register_keras_serializable

@register_keras_serializable(package="Custom", name="acc")
def acc(y_true, y_pred):
    YT = tf.cast(y_true > 0, tf.int8)
    YP = tf.cast(y_pred > 0, tf.int8)
    accuracy = tf.reduce_mean(tf.cast(YT == YP, tf.float32))
    return accuracy
# Load pre-trained RNN model
@st.cache_resource
def load_model():
    return tf.keras.models.load_model('AI.keras', custom_objects={
    'acc': acc,
})


In [25]:

model = load_model()

In [30]:
predicted_price = model.predict(np.array(Xl1.values[np.newaxis]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 375ms/step


In [31]:
np.array(Xl1.values)

array([[-1.41650350e-02,  1.46498335e-02, -1.80342700e-03,
         3.05243337e-02, -3.92171191e-02,  7.43446747e-03],
       [ 1.79087706e-02,  1.19611744e-02,  2.37588446e-02,
         2.24034287e-02, -1.72452209e-02,  1.19291256e-02],
       [ 3.87618101e-02,  4.68813725e-02,  4.11631316e-02,
         4.33552115e-02,  1.83806344e-01,  1.65225270e-02],
       [ 2.69234487e-03,  3.47980461e-04, -3.79634094e-03,
         1.01936121e-03, -4.21490700e-01,  1.48495433e-02],
       [ 2.26305894e-02, -1.28853064e-02,  1.17489907e-02,
        -3.17443586e-02,  4.08813237e-01, -3.95254047e-02],
       [-1.72996641e-01, -1.95375151e-01, -1.48594608e-01,
        -1.85946105e-01,  1.24975016e+00, -5.24748687e-02],
       [-2.42500467e-02, -3.86346867e-03,  4.66206064e-03,
         8.54973129e-02, -3.45424826e-01,  5.72724909e-02],
       [ 3.77798745e-02,  3.21653036e-02, -1.64918398e-02,
        -4.18756698e-02, -2.15865297e-01, -2.23830534e-02],
       [-2.72452874e-02, -1.63766378e-02, -1.500

In [34]:
predicted_price[0][0]

-0.0038487613

In [35]:
Xl1

,diff_n_o,diff_n_l,diff_n_h,diff_n_c,diff_n_v,diff_n_CO
1,-0.014165,0.014650,-0.001803,0.030524,-0.039217,0.007434
2,0.017909,0.011961,0.023759,0.022403,-0.017245,0.011929
3,0.038762,0.046881,0.041163,0.043355,0.183806,0.016523
4,0.002692,0.000348,-0.003796,0.001019,-0.421491,0.014850
5,0.022631,-0.012885,0.011749,-0.031744,0.408813,-0.039525
6,-0.172997,-0.195375,-0.148595,-0.185946,1.249750,-0.052475
7,-0.024250,-0.003863,0.004662,0.085497,-0.345425,0.057272
8,0.037780,0.032165,-0.016492,-0.041876,-0.215865,-0.022383
9,-0.027245,-0.016377,-0.015007,0.007651,-0.172967,0.012513
10,0.005509,0.009187,0.022544,-0.037435,-0.012200,-0.030431


In [ ]:
import yfinance as yf

# ระบุสัญลักษณ์หุ้นและวันที่ที่ต้องการดึงข้อมูล
ticker = "AAPL"  # ตัวอย่างหุ้น Apple
date = "2024-11-8"

# ดึงข้อมูลหุ้น
stock = yf.download(ticker, start=date, end=date)

# แสดงผลลัพธ์
print(stock)
